In [1]:
%pylab nbagg

Populating the interactive namespace from numpy and matplotlib


In [70]:
from tvb.simulator.lab import *
import h5py
import os
import matplotlib.pyplot as plt

import scipy 
import scipy.fftpack
import seaborn as sns

import scipy 
import scipy.fftpack
from scipy.fft import fft, ifft
from scipy import signal
from scipy.integrate import simps
from fooof import FOOOF
from scipy import stats

import pandas as pd
from statistics import stdev

In [3]:
os.chdir("C:/Users/celie/OneDrive - ugentbe/NOG NODIG/2de master/Masterproef/Wilson Cowan/DATA")

In [4]:
#import connectivity file to know which region corresponds to which index in the matrices
filename = "Connectivity.h5"

with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    regions_list = list(f['region_labels'])

regions = np.array(regions_list)

Keys: <KeysViewHDF5 ['areas', 'centres', 'cortical', 'hemispheres', 'orientations', 'region_labels', 'tract_lengths', 'weights']>


In [5]:
#import timeseries healthy: alpha power
mydir = "C:/Users/celie/OneDrive - ugentbe/NOG NODIG/2de master/Masterproef/Datasets/All_freqs"
    
filename = mydir + "/Time_Health.h5"
with h5py.File(filename, "r") as f:
    healthtime = f["time"]
    healthtime = np.array(healthtime)
    
filename = mydir + "/power_alpha_overall.h5"
with h5py.File(filename, "r") as f:
    alpha_all = f["data"]
    alpha_all = np.array(alpha_all)

filename = mydir + "/power_alpha_lDLPFC.h5"
with h5py.File(filename, "r") as f:
    alpha_left = f["data"]
    alpha_left = np.array(alpha_left)
    
filename = mydir + "/power_alpha_rDLPFC.h5"
with h5py.File(filename, "r") as f:
    alpha_right = f["data"]
    alpha_right = np.array(alpha_right)
    
filename = mydir + "/power_alpha_overall_depr.h5"
with h5py.File(filename, "r") as f:
    alpha_all_dep = f["data"]
    dep_all = np.array(alpha_all_dep)
    

filename = mydir + "/power_alpha_lDLPFC_depr.h5"
with h5py.File(filename, "r") as f:
    alpha_left_depr = f["data"]
    alpha_left_depr = np.array(alpha_left_depr)
    
filename = mydir + "/power_alpha_rDLPFC_depr.h5"
with h5py.File(filename, "r") as f:
    alpha_right_depr = f["data"]
    alpha_right_depr = np.array(alpha_right_depr)
    
    

filename = mydir + "/power_alpha_overall_HF.h5"
with h5py.File(filename, "r") as f:
    alpha_all_HF = f["data"]
    HF_all = np.array(alpha_all_HF)
    

filename = mydir + "/power_alpha_lDLPFC_HF.h5"
with h5py.File(filename, "r") as f:
    alpha_left_HF = f["data"]
    alpha_left_HF = np.array(alpha_left_HF)
    
filename = mydir + "/power_alpha_rDLPFC_HF.h5"
with h5py.File(filename, "r") as f:
    alpha_right_HF = f["data"]
    alpha_right_HF = np.array(alpha_right_HF)

filename = mydir + "/power_alpha_overall_LF.h5"
with h5py.File(filename, "r") as f:
    alpha_all_LF = f["data"]
    LF_all = np.array(alpha_all_LF)
    

filename = mydir + "/power_alpha_lDLPFC_LF.h5"
with h5py.File(filename, "r") as f:
    alpha_left_LF = f["data"]
    alpha_left_LF = np.array(alpha_left_HF)
    
filename = mydir + "/power_alpha_rDLPFC_LF.h5"
with h5py.File(filename, "r") as f:
    alpha_right_LF = f["data"]
    alpha_right_LF = np.array(alpha_right_HF)

In [44]:
import statistics
def EFsize(c0, c1):
    d = (mean(c0) - mean(c1)) / (sqrt((statistics.stdev(c0) ** 2 + statistics.stdev(c1) ** 2) / 2))
    return d

In [71]:
power_arr = numpy.zeros(shape = (6,5))
ef_arr = numpy.zeros(shape = (6,5))
means_left = numpy.zeros(shape = (4,2))

#alpha over left DLPFC
power_arr[0] = stats.ttest_ind(alpha_left, alpha_left_depr).pvalue #depr vs healthy 
power_arr[1] = stats.ttest_ind(alpha_left_LF, alpha_left).pvalue #LF vs healthy 
power_arr[2] = stats.ttest_ind(alpha_left_HF, alpha_left).pvalue #HF vs healthy 
power_arr[3] = stats.ttest_ind(alpha_left_LF, alpha_left_depr).pvalue #depr vs LF
power_arr[4] = stats.ttest_ind(alpha_left_HF, alpha_left_depr).pvalue #depr vs HF
power_arr[5] = stats.ttest_ind(alpha_left_HF,alpha_left_LF).pvalue #HF vs LF

for i in range(5):
    ef_arr[0,i] = EFsize(alpha_left[i], alpha_left_depr[i]) #depr vs healthy 
    ef_arr[1,i] = EFsize(alpha_left_LF[i], alpha_left[i]) #LF vs healthy 
    ef_arr[2,i] = EFsize(alpha_left_HF[i], alpha_left[i])#HF vs healthy 
    ef_arr[3,i] = EFsize(alpha_left_LF[i], alpha_left_depr[i]) #depr vs LF
    ef_arr[4,i] = EFsize(alpha_left_HF[i], alpha_left_depr[i]) #depr vs HF
    ef_arr[5,i] = EFsize(alpha_left_HF[i],alpha_left_LF[i]) #HF vs LF
    
means_left[0,0] = mean(alpha_left)
means_left[1,0] = mean(alpha_left_depr)
means_left[2,0] = mean(alpha_left_HF)
means_left[3,0] = mean(alpha_left_LF)
means_left[0,1] = stdev(alpha_left)
means_left[1,1] = stdev(alpha_left_depr)
means_left[2,1] = stdev(alpha_left_HF)
means_left[3,1] = stdev(alpha_left_LF)



new_column = ["Depr_health", "LF_health","HF_health","depr_LF","depr_HF","HF_LF"]
tuple = (power_arr, ef_arr, new_column)
power_arr_tot = np.column_stack(tuple)

file_name = mydir + "/Alpha_leftDLPFC.csv"
outcome = pd.DataFrame.from_records(power_arr_tot)
outcome.columns = ["p 8_9", "p 9_10", "p 10_11", "p 11_12", "p 8_12","ef 8_9", "ef 9_10", "ef 10_11", "ef 11_12", "ef 8_12", "comparison"]
outcome.to_csv(path_or_buf = file_name, index = False)

TypeError: can't convert type 'ndarray' to numerator/denominator

In [66]:
print(means_left)

[[1.71369649e-07 1.71369649e-07]
 [7.21824016e-08 7.21824016e-08]
 [1.88531545e-05 1.88531545e-05]
 [1.88531545e-05 1.88531545e-05]]


In [57]:
power_arr = numpy.zeros(shape = (6,5))
ef_arr = numpy.zeros(shape = (6,5))

means_all = numpy.zeros(shape = (4,2))

#alpha over all regions
power_arr[0] = stats.ttest_ind(alpha_all, dep_all).pvalue #depr vs healthy 
power_arr[1] = stats.ttest_ind(LF_all, alpha_all).pvalue #LF vs healthy 
power_arr[2] = stats.ttest_ind(HF_all, alpha_all).pvalue #HF vs healthy 
power_arr[3] = stats.ttest_ind(LF_all, dep_all).pvalue #depr vs LF
power_arr[4] = stats.ttest_ind(HF_all, dep_all).pvalue #depr vs HF
power_arr[5] = stats.ttest_ind(HF_all, LF_all).pvalue #HF vs LF

for i in range(5):
    ef_arr[0,i] = EFsize(alpha_all[i], dep_all[i]) #depr vs healthy 
    ef_arr[1,i] = EFsize(LF_all[i], alpha_all[i]) #LF vs healthy 
    ef_arr[2,i] = EFsize(HF_all[i], alpha_all[i])#HF vs healthy 
    ef_arr[3,i] = EFsize(LF_all[i], dep_all[i]) #depr vs LF
    ef_arr[4,i] = EFsize(HF_all[i], dep_all[i]) #depr vs HF
    ef_arr[5,i] = EFsize(HF_all[i],LF_all[i]) #HF vs LF
    
means_all[0,0] = mean(alpha_all)
means_all[1,0] = mean(dep_all)
means_all[2,0] = mean(HF_all)
means_all[3,0] = mean(LF_all)
means_all[0,1] = stdev(alpha_all)
means_all[1,1] = stdev(dep_all)
means_all[2,1] = stdev(HF_all)
means_all[3,1] = stdev(LF_all)

new_column = ["Depr_health", "LF_health","HF_health","depr_LF","depr_HF","HF_LF"]
tuple = (power_arr, ef_arr, new_column)
power_arr_tot = np.column_stack(tuple)

file_name = mydir + "/Alpha_all_regions.csv"
outcome = pd.DataFrame.from_records(power_arr_tot)
outcome.columns = ["p 8_9", "p 9_10", "p 10_11", "p 11_12", "p 8_12","ef 8_9", "ef 9_10", "ef 10_11", "ef 11_12", "ef 8_12", "comparison"]
outcome.to_csv(path_or_buf = file_name, index = False)

In [58]:
power_arr = numpy.zeros(shape = (6,5))
ef_arr = numpy.zeros(shape = (6,5))

means_right = numpy.zeros(shape = (4,2))

#alpha over right DLFPC
power_arr[0] = stats.ttest_ind(alpha_right, alpha_right_depr).pvalue #depr vs healthy 
power_arr[1] = stats.ttest_ind(alpha_right_LF, alpha_right).pvalue #LF vs healthy 
power_arr[2] = stats.ttest_ind(alpha_right_HF, alpha_right).pvalue #HF vs healthy 
power_arr[3] = stats.ttest_ind(alpha_right_LF, alpha_right_depr).pvalue #depr vs LF
power_arr[4] = stats.ttest_ind(alpha_right_HF, alpha_right_depr).pvalue #depr vs HF
power_arr[5] = stats.ttest_ind(alpha_right_HF,alpha_right_LF).pvalue #HF vs LF

for i in range(5):
    ef_arr[0,i] = EFsize(alpha_right[i], alpha_right_depr[i]) #depr vs healthy 
    ef_arr[1,i] = EFsize(alpha_right_LF[i], alpha_right[i]) #LF vs healthy 
    ef_arr[2,i] = EFsize(alpha_right_HF[i], alpha_right[i])#HF vs healthy 
    ef_arr[3,i] = EFsize(alpha_right_LF[i], alpha_right_depr[i]) #depr vs LF
    ef_arr[4,i] = EFsize(alpha_right_HF[i], alpha_right_depr[i]) #depr vs HF
    ef_arr[5,i] = EFsize(alpha_right_HF[i],alpha_right_LF[i]) #HF vs LF
    

means_right[0,0] = mean(alpha_right)
means_right[1,0] = mean(alpha_right_depr)
means_right[2,0] = mean(alpha_right_HF)
means_right[3,0] = mean(alpha_right_LF)
means_right[0,1] = stdev(alpha_right)
means_right[1,1] = stdev(alpha_right_depr)
means_right[2,1] = stdev(alpha_right_HF)
means_right[3,1] = stdev(alpha_right_LF)

new_column = ["Depr_health", "LF_health","HF_health","depr_LF","depr_HF","HF_LF"]
tuple = (power_arr, ef_arr, new_column)
power_arr_tot = np.column_stack(tuple)

file_name = mydir + "/Alpha_rightDLPFC.csv"
outcome = pd.DataFrame.from_records(power_arr_tot)
outcome.columns = ["p 8_9", "p 9_10", "p 10_11", "p 11_12", "p 8_12","ef 8_9", "ef 9_10", "ef 10_11", "ef 11_12", "ef 8_12", "comparison"]
outcome.to_csv(path_or_buf = file_name, index = False)

In [69]:
power_arr = numpy.zeros(shape = (6,5))
ef_arr = numpy.zeros(shape = (6,5))
means_as = numpy.zeros(shape = (4,2))

#alpha asymmetry values 
depr_as = alpha_left_depr/alpha_right_depr
health_as = alpha_left/alpha_right
HF_as = alpha_left_HF/alpha_right_HF
LF_as = alpha_left_LF/alpha_right_LF


power_arr[0] = stats.ttest_ind(depr_as, health_as).pvalue #depr vs healthy 
power_arr[1] = stats.ttest_ind(LF_as, health_as).pvalue #LF vs healthy 
power_arr[2] = stats.ttest_ind(HF_as, health_as).pvalue #HF vs healthy 
power_arr[3] = stats.ttest_ind(LF_as, depr_as).pvalue #depr vs LF
power_arr[4] = stats.ttest_ind(HF_as, depr_as).pvalue #depr vs HF
power_arr[5] = stats.ttest_ind(HF_as,LF_as).pvalue #HF vs LF


for i in range(5):
    ef_arr[0,i] = EFsize(health_as[i], depr_as[i]) #depr vs healthy 
    ef_arr[1,i] = EFsize(LF_as[i], health_as[i]) #LF vs healthy 
    ef_arr[2,i] = EFsize(HF_as[i], health_as[i])#HF vs healthy 
    ef_arr[3,i] = EFsize(LF_as[i], depr_as[i]) #depr vs LF
    ef_arr[4,i] = EFsize(HF_as[i], depr_as[i]) #depr vs HF
    ef_arr[5,i] = EFsize(HF_as[i],LF_as[i]) #HF vs LF
    

means_as[0,0] = mean(health_as)
means_as[1,0] = mean(depr_as)
means_as[2,0] = mean(HF_as)
means_as[3,0] = mean(LF_as)
means_as[0,1] = stdev(health_as)
means_as[1,1] = stdev(depr_as)
means_as[2,1] = stdev(HF_as)
means_as[3,1] = stdev(LF_as)

new_column = ["Depr_health", "LF_health","HF_health","depr_LF","depr_HF","HF_LF"]
tuple = (power_arr, ef_arr, new_column)
power_arr_tot = np.column_stack(tuple)

file_name = mydir + "/Alpha_asymmetry.csv"
outcome = pd.DataFrame.from_records(power_arr_tot)
outcome.columns = ["p 8_9", "p 9_10", "p 10_11", "p 11_12", "p 8_12","ef 8_9", "ef 9_10", "ef 10_11", "ef 11_12", "ef 8_12", "comparison"]
outcome.to_csv(path_or_buf = file_name, index = False)



NameError: name 'stdev' is not defined